In [2]:
import pandas as pd
import json
import numpy as np

In [10]:
samples = []
with open('./data.jsonl', 'r') as jsonl:
    for line in jsonl.readlines():
        try:
            item = json.loads(line)
            item.update(item['label'])
            item.pop('label')
            for key in item.keys():
                if item[key] == 'none':
                    item[key] = None
            samples.append(item)
        except:
            ...
df = pd.DataFrame.from_dict(samples)
df

,prompt,role,task,context,input,output_requirements,constraints,example
0,Write a Python function that reverses a string.,None,Write a Python function that reverses a string.,None,None,None,None,None
1,Tôi muốn bạn đóng vai một giáo viên tiếng Anh ...,giáo viên tiếng Anh,sửa đoạn văn,None,'He go to school every day.',None,None,None
2,Summarize the following article in 3 bullet po...,None,Summarize the following article in 3 bullet po...,None,Climate change is affecting global weather pat...,3 bullet points,None,None
3,Generate a SQL query to select all users older...,None,Generate a SQL query,None,users table with age column,select all users older than 30,None,None
4,"ê bot, giúp tao viết đoạn code JavaScript in r...",None,viết đoạn code JavaScript in ra số từ 1 đến 10,None,None,None,None,None
...,...,...,...,...,...,...,...,...
3186,Viết một đoạn mã C để kiểm tra số hoàn hảo lớn...,None,Viết mã C kiểm tra số hoàn hảo lớn hơn 100000,None,None,Đoạn mã C,None,int nextPerfect(int n) { ... }
3187,What is the capital of Finland?,None,Answer capital of Finland,None,None,None,None,None
3188,"ê bot, tao cần code Java để tạo file backup, n...",None,Viết code Java tạo file backup,None,None,Kiểm tra lỗi ghi file,None,None
3189,List 19 types of reptiles.,None,List types of reptiles,None,None,19 types,None,None


In [22]:
def template(prompt: str):
    return '''
    You are an expert prompt analyst.
    
    Your task is to extract the structural components of the user’s prompt.  
    Break the prompt into exactly **7 components**, following the definitions below:
    
    1. role — The role or persona the user wants the model to adopt.  
    2. task — The main action or objective the user wants the model to perform.  
    3. context — Background information or situational details that help explain *why* or *in what situation* the task is performed.  
    4. input — The data, text, code, or content that the user wants the model to operate on directly.  
    5. output_requirements — Requirements about the format, style, structure, tone, or constraints on the output.  
    6. constraints — Rules or limitations the model must follow.  
    7. example — Any example provided by the user that illustrates the expected output.
    
    ### IMPORTANT RULES
    - If a component is *not explicitly present*, set its value to `"none"`.  
    - Do NOT hallucinate. Only extract what is truly there.  
    - Follow the JSON schema below exactly. No extra fields.  
    - Output must be **valid JSON**, no explanation, no comments.
    
    ### JSON Schema
    {
      "role": "string or 'none'",
      "task": "string or 'none'",
      "context": "string or 'none'",
      "input": "string or 'none'",
      "output_requirements": "string or 'none'",
      "constraints": "string or 'none'",
      "example": "string or 'none'"
    }
    
    ### USER PROMPT TO ANALYZE
    %s
    ''' % prompt

In [30]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    'google/gemma-3-270m-it',
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained('google/gemma-3-270m-it')

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use mps


In [34]:
messages = [
    {"role": "user", "content": template(prompt=df.loc[0]['prompt'])},
]

prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) #type:ignore
outputs = pipe(prompt, max_new_tokens=512, disable_compile=True)
print(f'Prompt\n{prompt}')
print(f'Expected value\n{df.loc[0]}')
print(f'Generated\n{outputs[0]["generated_text"][len(prompt):].strip()}')

Prompt
<bos><start_of_turn>user
You are an expert prompt analyst.

    Your task is to extract the structural components of the user’s prompt.  
    Break the prompt into exactly **7 components**, following the definitions below:

    1. role — The role or persona the user wants the model to adopt.  
    2. task — The main action or objective the user wants the model to perform.  
    3. context — Background information or situational details that help explain *why* or *in what situation* the task is performed.  
    4. input — The data, text, code, or content that the user wants the model to operate on directly.  
    5. output_requirements — Requirements about the format, style, structure, tone, or constraints on the output.  
    6. constraints — Rules or limitations the model must follow.  
    7. example — Any example provided by the user that illustrates the expected output.

    ### IMPORTANT RULES
    - If a component is *not explicitly present*, set its value to `"none"`.  
  